In [2]:
from tkinter import *
import tkinter as tk
from tkinter import ttk
import sqlite3
import os.path

#функция создания БД
def click_btn1():
    windows1 = tk.Tk()
    windows1.geometry('500x110')
    windows1.title=('Создадим базу данных sqlite3')
    label1=tk.Label(windows1,text='База данных создается подождите...',font=("Arial",20))
    label1.grid(column=0,row=0)
    file_path = "BD_avto.db"
    if os.path.exists(file_path):  os.remove(file_path)
    #вначале грохним базу если она есть, чтобы не мешала
    conn = sqlite3.connect("BD_avto.db")
    c = conn.cursor()
    c.execute('''
     CREATE TABLE "avto" (
	    "id"	INTEGER NOT NULL UNIQUE,
    	"VIN"	TEXT,
	    "YEAR"	INTEGER,
	    "posivnoy"	NUMERIC,
	    "model"	TEXT NOT NULL,
	    "color"	TEXT NOT NULL,
	    PRIMARY KEY("VIN")
        )
    ''')
    c.execute('''
      CREATE TABLE "drivers" (
    	"id"	INTEGER NOT NULL UNIQUE,
    	"FIO"	TEXT,
    	"BirthDay"	TEXT,
    	"PlaceBG"	TEXT,
    	"Foto"	BLOB,
    	PRIMARY KEY("id" AUTOINCREMENT)
      )
        ''')
    c.execute('''
    CREATE TABLE "ekipag" (
    	"id"	INTEGER NOT NULL UNIQUE,
    	"id_avto"	INTEGER NOT NULL,
    	"id_driver"	INTEGER NOT NULL,
    	"Data_BEG"	TEXT DEFAULT '',
    	"Data_END"	TEXT DEFAULT '',
    	PRIMARY KEY("id" AUTOINCREMENT),
    	FOREIGN KEY("id_avto") REFERENCES "avto"("id"),
	    FOREIGN KEY("id_driver") REFERENCES "drivers"("id")
    )
        ''')

    conn.commit()
    conn.close()
    label2=tk.Label(windows1,text='База пересоздана и она пустая',font=("Arial",20))
    label2.grid(column=0,row=2)
    btn1=tk.Button(windows1,text='Закрыть',command=windows1.destroy)
    btn1.grid(column=0, row=3)


#заполнение тестовыми данными
def click_btn2():
    windows2 = tk.Tk()
    windows2.geometry('500x110')
    windows2.title=('Наполним данными БД тестовыми')
    label1=tk.Label(windows2,text='База данных наполняется...',font=("Arial",20))
    label1.grid(column=0,row=0)
    #наполним машинами, главное у машины это VIN ПТС, СРки это доп документы реализовывать надо через доп таблицы
    avto_dat=[]
    avto_dat.append((1,'XW8ZZZCKZNG008690',2022,100,'Kia Rio','белый'))
    avto_dat.append((2,'Z8NAJL10059588017',2021,200,'Nissan Almera','серый'))
    avto_dat.append((3,'Z8NAJL10061568726',2020,300,'Toyota Camry','черный'))
    conn = sqlite3.connect("BD_avto.db")
    c = conn.cursor()
    for i in avto_dat:
        c.execute("insert into avto(id,vin,year,posivnoy,model,color) values( ?,?,?,?,?,?)",i)
    btn3 = tk.Button(windows2, text='Закрыть', command=windows2.destroy)
    btn3.grid(column=0, row=3)
    conn.commit()
    #водителями наполним, без паспортных без ВУ, на это нужны доп таблицы
    driver_dat=[]
    driver_dat.append((1,'Иванов Иван Иванович','13.07.1978','г. Барнаул'))
    driver_dat.append((2,'Журавлев Эдуард Иванович','20.04.1977','г. Новосибирск'))
    driver_dat.append((3,'Путин Владимир Эдуардов оглы','01.03.2000','г. Москва'))
    for i in driver_dat:
        c.execute("insert into drivers(id,FIO,BirthDay,PlaceBG) values(?,?,?,?)",i)
    conn.commit()
    #посадим водителей на автомобили это называется экипаж
    ekipaz_dat=[]
    ekipaz_dat.append((1,1,1,'01.01.2022','28.04.2022'))
    ekipaz_dat.append((2,1,2,'01.05.2022',''))
    ekipaz_dat.append((3,3,1,'29.04.2022',''))

    for i in ekipaz_dat:
        c.execute("insert into ekipag(id,id_avto,id_driver,Data_BEG,Data_END) values(?,?,?,?,?)",i)
    conn.commit()
    conn.close()
    label2=tk.Label(windows2,text='База наполнена',font=("Arial",20))
    label2.grid(column=0,row=2)
    btn1=tk.Button(windows2,text='Закрыть',command=windows2.destroy)
    btn1.grid(column=0, row=3)



#выведем данные в окно на грид
def btn_SQL(BD_sql,title,sql_txt,columns):
    windows3 = tk.Tk()

    windows3.title=(title)
    # определяем столбцы

    conn = sqlite3.connect(BD_sql)
    c = conn.cursor()
    c.execute(sql_txt)

    data = (row for row in c.fetchall())
    tbl=ttk.Treeview(windows3, show="headings", selectmode="browse")
    tbl["columns"] = columns
    tbl["displaycolumns"] = columns
    # названия колонок в шапку закинем и по ширине столбца сделаем ширину колонки
    w=0 # ширина окна
    for i in columns:
        tbl.heading(i,text=i, anchor=W)
        tbl.column(i, stretch=NO, width=len(i)*10)
        w=w+len(i)*10
    #ширина окна будет зависить от ширины всех столбцов
    windows3.geometry(repr(w)+'x400')
    for i in data:
        tbl.insert('', tk.END, values=tuple(i))

    scrolltable = tk.Scrollbar(windows3, command=tbl.yview)
    tbl.configure(yscrollcommand=scrolltable.set)
    scrolltable.pack(side=tk.RIGHT, fill=tk.Y)
    tbl.pack(fill=BOTH, expand=1)

    c.close()
    btn1=tk.Button(windows3,text='Закрыть',command=windows3.destroy)
    btn1.pack()

def click_btn3():
    btn_SQL("BD_avto.db",
            "Таблица Автомобили",
            "select id,model,color,VIN,YEAR,posivnoy from avto",
            ("ИД  ", "Модель      ", "Цвет     ", "VIN            ", "Год выпуска", "Позывной"))

def click_btn4():
    btn_SQL("BD_avto.db",
            "Таблица Водители",
            "select id,FIO,BirthDay,PlaceBG from drivers",
            ("ИД  ", "ФИО                    ", "День Рождения     ", "Место Рождения            "))

def click_btn5():
    btn_SQL("BD_avto.db",
            "Таблица Экипажи",
            "select d.FIO, a.posivnoy, a.model, a.color, a.VIN, e.Data_BEG, e.Data_END from ekipag e, drivers d, avto a where e.id_avto=a.id and e.id_driver=d.id",
            ( "ФИО                    ", "Позывной", "Модель     ","Цвет   ","VIN                ","Дата начала аренды авто", "Дата окончания аренды авто"))


windows = tk.Tk()
windows.geometry('250x200+200+200')
windows.title("Лабораторная работа 4 Токарев Александр Васильевич")

label1=tk.Label(windows,text="База данных Авто, Водители, Экипажи",width=30, height=2)
label1.grid(column=0,row=0)

btn1=tk.Button(windows,text="Создадим БД пустую",command=click_btn1)
btn1.grid(column=0,row=1)

btn2=tk.Button(windows,text="Наполним данными",command=click_btn2)
btn2.grid(column=0,row=2)

btn3=tk.Button(windows,text="Таблица Автомобили",command=click_btn3)
btn3.grid(column=0,row=4)

btn4=tk.Button(windows,text="Таблица Водители",command=click_btn4)
btn4.grid(column=0,row=6)

btn5=tk.Button(windows,text="Таблица Экипажи",command=click_btn5)
btn5.grid(column=0,row=8)

windows.mainloop()
